In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys


In [3]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time', 'precipIntensityMaxTime']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'precipIntensityMaxTime1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh

In [4]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/BerkeleyResults/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [10]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    print(num_categorical_cols)
    print(input_size)
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [11]:
def train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(categorical_embedding_sizes, train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.L1Loss()
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_c_t, train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_c_t, test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('2 Layers Architecture')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

2 Layers Architecture
MAE =  4.293805360794067 r2 =  0.4860425043714171 SE =  0.03157491319792726 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.293805360794067

MAE =  4.362303161621094 r2 =  0.4276671815952612 SE =  0.03015904893456131 lr =  0.001 hz =  50 dr =  0.1
MAE =  4.8403770446777346 r2 =  0.45840950458329066 SE =  0.04389269008429214 lr =  0.0001 hz =  50 dr =  0.1
MAE =  4.29057936668396 r2 =  0.5000480344597737 SE =  0.03182607586086987 lr =  0.01 hz =  60 dr =  0.1
Best so far 4.29057936668396

MAE =  4.40277247428894 r2 =  0.4356179989112377 SE =  0.02370046153116595 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.767365217208862 r2 =  0.4653958025845588 SE =  0.04345683453615037 lr =  0.0001 hz =  60 dr =  0.1
MAE =  4.1839213371276855 r2 =  0.4916072800723844 SE =  0.02739924138563103 lr =  0.01 hz =  70 dr =  0.1
Best so far 4.1839213371276855

MAE =  4.386030101776123 r2 =  0.4491082933783086 SE =  0.025001883572804872 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.655900573730468 r2 

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture
MAE =  4.305554533004761 r2 =  0.4596570214763929 SE =  0.02869728433026032 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.305554533004761

MAE =  4.404421424865722 r2 =  0.38486107360705796 SE =  0.022848938289328345 lr =  0.001 hz =  50 dr =  0.1
MAE =  4.809010028839111 r2 =  0.4103305339746477 SE =  0.040629125886896676 lr =  0.0001 hz =  50 dr =  0.1
MAE =  4.236850738525391 r2 =  0.5206707237398611 SE =  0.034180677492352436 lr =  0.01 hz =  60 dr =  0.1
Best so far 4.236850738525391

MAE =  4.414889860153198 r2 =  0.44495797403861115 SE =  0.023743774743828445 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.642010116577149 r2 =  0.4768680027330888 SE =  0.036076959538924994 lr =  0.0001 hz =  60 dr =  0.1
MAE =  4.2669891834259035 r2 =  0.5151760560401499 SE =  0.030177630890864372 lr =  0.01 hz =  70 dr =  0.1
MAE =  4.399434518814087 r2 =  0.4857386898839501 SE =  0.027812222049757068 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.768790864944458 r2 =  0.4449781121736559 S

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))


    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  4.268816089630127 r2 =  0.5153649744712687 SE =  0.029582582741269757 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.268816089630127

MAE =  4.390058469772339 r2 =  0.4894013196988616 SE =  0.029596396991820078 lr =  0.001 hz =  50 dr =  0.1
MAE =  4.6331787109375 r2 =  0.4700593382462522 SE =  0.03785395019647295 lr =  0.0001 hz =  50 dr =  0.1
MAE =  4.244509840011597 r2 =  0.49482345511454245 SE =  0.027351813883940146 lr =  0.01 hz =  60 dr =  0.1
Best so far 4.244509840011597

MAE =  4.34231390953064 r2 =  0.47415135089374916 SE =  0.027816977959856264 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.756696462631226 r2 =  0.3955325313163808 SE =  0.04480990923346564 lr =  0.0001 hz =  60 dr =  0.1
MAE =  4.176179122924805 r2 =  0.5178221674806261 SE =  0.02684253981553707 lr =  0.01 hz =  70 dr =  0.1
Best so far 4.176179122924805

MAE =  4.37139663696289 r2 =  0.48287639965902773 SE =  0.028239700736513763 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.7584795475006105 